In [1]:
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

from src.dataset import read_lang_dataset, tokenize_dataset, get_vocab_mapping

In [2]:
from src.dataset import gateways
gateways[0].jvm.Tokenizer.tokenizeCsharp("""
// Licensed to the .NET Foundation under one or more agreements.
// The .NET Foundation licenses this file to you under the MIT license.
// See the LICENSE file in the project root for more information.

using System.Linq;
using Microsoft.CodeAnalysis.CSharp.Symbols;

namespace Microsoft.CodeAnalysis.CSharp
{
    internal static class CSharpCompilationExtensions
    {
        int member;
        internal static bool IsFeatureEnabled(this CSharpCompilation compilation, MessageID feature)
        {
            var a = 5;
            int b = 0x5 + a;
            var tst = "abc" + 'd';
            var t = 56f + member;
            return null;
        }
    }
}
""")

['using', 'System', '.', 'Linq', ';', 'using', 'Microsoft', '.', 'CodeAnalysis', '.', 'CSharp', '.', 'Symbols', ';', 'namespace', 'Microsoft', '.', 'CodeAnalysis', '.', 'CSharp', '{', 'internal', 'static', 'class', 'CSharpCompilationExtensions', '{', 'int', 'VARIABLE', ';', 'internal', 'static', 'bool', 'IsFeatureEnabled', '(', 'this', 'CSharpCompilation', 'VARIABLE', ',', 'MessageID', 'VARIABLE', ')', '{', 'var', 'VARIABLE', '=', 'INT_LITERAL', ';', 'int', 'VARIABLE', '=', 'INT_LITERAL', '+', 'VARIABLE', ';', 'var', 'VARIABLE', '=', 'STRING_LITERAL', '+', 'STRING_LITERAL', ';', 'var', 'VARIABLE', '=', 'FLOAT_LITERAL', '+', 'VARIABLE', ';', 'return', 'null', ';', '}', '}', '}', '<EOF>']

In [3]:
gateways[0].jvm.Tokenizer.tokenizeCpp("""
#include <iostream>
using namespace std;

int main()
{
    int a = 5, b = 10, temp;

    cout << "Before swapping." << endl;
    cout << "a = " << a << ", b = " << b << endl;

    temp = a;
    a = b;
    b = temp;

    cout << "\nAfter swapping." << endl;
    cout << "a = " << a << ", b = " << b << endl;

    return 0;
}
""")

['using', 'namespace', 'std', ';', 'int', 'main', '(', ')', '{', 'int', 'VARIABLE', '=', 'INT_LITERAL', ',', 'VARIABLE', '=', 'INT_LITERAL', ',', 'VARIABLE', ';', 'cout', '<', '<', 'STRING_LITERAL', '<', '<', 'endl', ';', 'cout', '<', '<', 'STRING_LITERAL', '<', '<', 'VARIABLE', '<', '<', 'STRING_LITERAL', '<', '<', 'VARIABLE', '<', '<', 'endl', ';', 'VARIABLE', '=', 'VARIABLE', ';', 'VARIABLE', '=', 'VARIABLE', ';', 'VARIABLE', '=', 'VARIABLE', ';', 'cout', '<', '<', 'After', 'using', 'namespace', 'std', ';', 'int', 'VARIABLE', '(', ')', '{', 'int', 'VARIABLE', '=', 'INT_LITERAL', ',', 'VARIABLE', '=', 'INT_LITERAL', ',', 'VARIABLE', ';', 'cout', '<', '<', 'STRING_LITERAL', '<', '<', 'endl', ';', 'cout', '<', '<', 'STRING_LITERAL', '<', '<', 'VARIABLE', '<', '<', 'STRING_LITERAL', '<', '<', 'VARIABLE', '<', '<', 'endl', ';', 'VARIABLE', '=', 'VARIABLE', ';', 'VARIABLE', '=', 'VARIABLE', ';', 'VARIABLE', '=', 'VARIABLE', ';', 'cout', '<', '<', 'After', 'swapping', '.', 'cout', '<', '<'

In [4]:
csharp_df = read_lang_dataset("data/csharp_codes.db")
csharp_df

,language,code
0,C#,using System;\nusing System.Collections.Generi...
1,C#,using System;\nusing System.Collections.Generi...
2,C#,using System.Collections.Generic;\nusing Syste...
3,C#,using System;\nusing System.Windows;\nusing Sy...
4,C#,using System;\nusing System.Collections.Generi...
...,...,...
1108,C#,using System;\nusing System.Collections.Generi...
1109,C#,// <auto-generated />\nusing System;\nusing Mi...
1110,C#,"#pragma checksum ""C:\Users\gault\Desktop\Effic..."
1111,C#,using System;\nusing System.Collections.Generi...


In [4]:
sample = csharp_df.sample(2)
sample

,language,code
454,C#,// This file is part of the OWASP O2 Platform ...
158,C#,using System;\nusing System.Collections.Generi...


In [5]:
tokenized_sample = tokenize_dataset(sample)
tokenized_sample

100%|██████████| 2/2 [00:02<00:00,  1.24s/it]


,language,code
454,C#,"[using, System, ;, using, System, ., Linq, ;, ..."
158,C#,"[using, System, ;, using, System, ., Collectio..."


In [7]:
' '.join(tokenized_sample.code.iloc[1][:550])

'using System ; using System . Collections . Generic ; using System . Text ; using System . Web . UI ; using CMS . Automation ; using CMS . Helpers ; using CMS . Base ; using CMS . WorkflowEngine ; using CMS . ExtendedControls . ActionsConfig ; using CMS . Controls ; using CMS . ExtendedControls ; using CMS . WorkflowEngine . Definitions ; using CMS . Membership ; using CMS . FormControls ; using CMS . DataEngine ; public partial class CMSModules_Automation_Controls_AutomationMenu : BaseEditMenu { protected NextStepAction VARIABLE = null ; protected PreviousStepAction VARIABLE = null ; protected HeaderAction VARIABLE = null ; protected HeaderAction VARIABLE = null ; protected NextStepAction VARIABLE = null ; private WorkflowStepInfo VARIABLE ; private CMSAutomationManager VARIABLE ; private bool ? VARIABLE ; private string VARIABLE ; public override HeaderActions HeaderActions { get { return menu ; } } public override bool AllowSave { get { if ( VARIABLE == null ) { return AutomationMa

In [10]:
np.random.seed(42)
sample = data.sample(3)
for i, language, code in sample.itertuples():
    print(f"###### Index {i:7} ######")
    print(code)
    print(f"###########################")

NameError: name 'data' is not defined

In [ ]:
sample = tokenize_dataset(sample)
sample

100%|██████████| 3/3 [00:01<00:00,  2.06it/s]


,language,code
77165,C++,[]
307910,C++,[]
79469,C++,[]


In [ ]:
for language, code in sample.itertuples(False):
    print(code)

[]
[]
[]


In [ ]:
tokenize_dataset(data)

100%|██████████| 327126/327126 [04:00<00:00, 1357.60it/s]


,language,code
0,C++,"[http, :, Unless, required, http, Unless, requ..."
1,C++,[<EOF>]
2,C++,[]
3,C++,"[AddError, (, INT_LITERAL, ,, INT_LITERAL, ,, ..."
4,C++,[]
...,...,...
327121,C++,[]
327122,C++,[<EOF>]
327123,C++,[]
327124,C++,"[assert, (, not_found, ==, answer1, ), ;, answ..."


In [ ]:
datasets = [read_dataset(dataset_path, language) for language in ["Python", "Go", "C", "C++"]]
datasets = pd.concat(datasets, ignore_index=True)
datasets

,language,code
0,Python,"version = '7'\nhtml_title = ""Guzzle Documentat..."
1,Python,"# Path to a touch icon\n # ""touch_icon""..."
2,Python,"""base_url"": ""http://guzzlephp.org""\n\n ..."
3,Python,from tensorflow.python.estimator.model_fn impo...
4,Python,self.assertFalse(gfile.Exists('ram://exist...
...,...,...
1546444,C++,* * Worst Time Complexity O(log n)\n * * Best...
1546445,C++,/**\n * \file\n * \brief [Interpolation\n * se...
1546446,C++,std::cin >> n;\n\n int *array = new int...
1546447,C++,assert(not_found == answer1);\n // Test...


In [ ]:
tokenized = tokenize_dataset(datasets)
tokenized

 54%|█████▍    | 176115/327126 [02:03<01:46, 1422.12it/s]


KeyboardInterrupt: 

In [ ]:
words2int, int2word = get_vocab_mapping(tokenized)

: 